In [1]:
from algs_lib import *
from scipy import stats

In [3]:
all_data = {}
for data in ['iris', 'rice', 'bean']:
    fname_local = f'mean_dists/global_{data}_mean_dist.pkl'
    if data not in all_data:
        all_data[data] = {}
    all_data[data]['local'] = pickle.load(open(fname_local, 'rb'))['individual']
    fname_global = f'mean_dists/global_{data}_mean_dist.pkl'
    global_data = pickle.load(open(fname_global, 'rb'))
    all_data[data]['global'] = global_data['global']
    all_data[data]['dp'] = global_data['dp']

In [4]:
for k in ['global']:
    for data in ['iris', 'rice', 'bean']:
        for mi in [1/4, 1/16, 1/64]:
            aniso = all_data[data][k][mi][1]
            iso = all_data[data][k][mi][2]
            print(iso/aniso)

1.137818261290776
1.1131056516502416
1.1270197063430685
1.617984159016757
1.6818536655933851
1.6865962895843374
2.087584400794275
2.192632896719595
2.1554819451373928
